Outline

1. Load data 

2. Split it into train and test set, define target

3. Feature engineering: 

   - missing data: median for numerical variables that are not really gaussian-distributed and mode for categorical
   - discretization: normalisation because of high rate of outliers
   - categorical encoding: OneHotEncoder because most categorical variables have few labels
4. algorithms: 
   - model training, model selection
   - model evaluation:


In [30]:
# import librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import category_encoders as ce
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

1. Load data

In [4]:
data = pd.read_csv(r'titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# remove irrelevant columns
data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=0, inplace=True)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


2. Train and Test sets

In [15]:
# Target and predictive variables
X = data.drop(data.columns[0], axis=1)
y = data[data.columns[0]]
X.shape, y.shape

((891, 7), (891,))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
test_size = 0.2,
random_state = 9)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

3. Feature Engineering

In [18]:
# Numerical and categarical variables
cat_vars = [var for var in X.columns if X[var].dtype=='O']
cat_vars = cat_vars + ['Pclass']
num_vars =[var for var in X.columns if var not in cat_vars and var !='Survived']
print(cat_vars)
num_vars

['Sex', 'Embarked', 'Pclass']


['Age', 'SibSp', 'Parch', 'Fare']

In [24]:
# Pipeline for numerical variables : fillna with median and normalize data
num_pip = make_pipeline(
    SimpleImputer(strategy = 'median'),
    MinMaxScaler()
)

In [27]:
# Pipeline for categorical variables : fillna with mode and use oneHotEncoder
cat_pip = make_pipeline(
    SimpleImputer(strategy = 'most_frequent'),
    OneHotEncoder()
)

In [31]:
# Build the feature transformation pipeline
feature_trans_pip = make_column_transformer(
    num_pip,
    cat_pip
)